_[NOTE] **google radar search** deprecated as of July 16 2018, using **nearby search & gmplot** instead_

# Business Location Analysis



<div style="text-align: right"> first drafted on 2016-07-11 </div>
<div style="text-align: right"> last updated on 2018-08-07 </div>

*It is a critical question to many business where to open and operate its offline spaces. Especially when a business is pursuing an expansion to places not so familiar to, researching and making a decision on where to locate can be a very difficult problem. I have had a similar concerns as a strategist, and came up with a simple alternative that helped me to make reasonable data-driven decision making.*

The logic is simple;
 
<br> 
<b>&nbsp;&nbsp;&nbsp;&nbsp;*'You want to do business from where your customers are.'*</b>


<br>
In order to find such a location, the following procedure has been taken from this analysis:

1. Define customers - Define sets of customers, which the business is targeting to. The purpose of business, marketing, and strategy all come in to play. If appripriate, apply STP to each set of customer.
2. Define sets - For each heterogeneous set of customers, allocation of brand or landmark that can represent characteristics of the set. It can be popular brand(shop), public places, or anything that can be searched on google map. More specific, the better.
3. Find on google - Google web API provides geocode for your search query. Geocode for each keyword location can be retrieved.
4. Plot on map - Now that each location information is given, plot heatmap layer on google map with the help of package 'gmaps'. It can be done separately by each set of customers, or as a whole.


- please refer to https://developers.google.com/maps/documentation/javascript/places?hl=ko#place_searches for more information on Google place API
<br>
___
For example, *a restaurant franchise seeking an international expansion* into London, UK may take the following procedure:

1. Define customers
 - Curious customers seeking for new things to try with fellows, with mid-upper range income to spare on dinning experience.
2. Define sets
 - Restaurant(restaurant) 
 - Pioneers (university, waterstones, starbucks)
 - Tourist (hotel, tourist attraction) 
 - Hang out (cinema, club)
3. Find on google
4. plot on map
___

The libraries used for this project are as follows;

- *re* for pattern & string process

- *gmplot* for Google maps API

- *urllib* for Google API and data crawling

- ~~gmaps for Google maps API~~

In [2]:
from urllib.request import urlopen
import gmplot
from IPython.display import IFrame
import re

Define target city and retrieve geocode using google api.

In [1]:
city = 'London'

city_url = 'https://maps.googleapis.com/maps/api/geocode/json?address='
key = "YOUR KEY HERE"

In [3]:
url = urlopen(city_url+city+key)
tmp = url.readlines()
for i in range(len(tmp)):
    if '"location"' in str(tmp[i]):
        city_coord =(float(re.sub('[a-z\ ":\',\\\]*','',str(tmp[i+1]))), float(re.sub('[a-z\ ":\',\\\]*','',str(tmp[i+2]))))
str(city_coord)[1:-1]

'51.5073509, -0.1277583'

If you look into the data retrieved, it contains a number of location informations as follows;

In [16]:
tmp

[b'{\n',
 b'   "results" : [\n',
 b'      {\n',
 b'         "address_components" : [\n',
 b'            {\n',
 b'               "long_name" : "London",\n',
 b'               "short_name" : "London",\n',
 b'               "types" : [ "locality", "political" ]\n',
 b'            },\n',
 b'            {\n',
 b'               "long_name" : "London",\n',
 b'               "short_name" : "London",\n',
 b'               "types" : [ "postal_town" ]\n',
 b'            },\n',
 b'            {\n',
 b'               "long_name" : "Greater London",\n',
 b'               "short_name" : "Greater London",\n',
 b'               "types" : [ "administrative_area_level_2", "political" ]\n',
 b'            },\n',
 b'            {\n',
 b'               "long_name" : "England",\n',
 b'               "short_name" : "England",\n',
 b'               "types" : [ "administrative_area_level_1", "political" ]\n',
 b'            },\n',
 b'            {\n',
 b'               "long_name" : "United Kingdom",\n',
 b'   

___

Repeat the similar procedure for customer sets, within 15km radius of the target, London for instance.

The coordinates will be stored in a list in a dictionary under the 'Keyword'.
___
- keywords for London market

In [4]:
restaurant = {'id': 'restaurant'}
restaurant['key'] = ['restaurant']

pioneer = {'id': 'pioneer'}
pioneer['key'] = ['university', 'waterstones', 'starbucks']

tourist = {'id': 'tourist'}
tourist['key'] = ['hotel', 'attraction']

hang_out = {'id': 'hang_out'}
hang_out['key'] = ['cinema', 'club']

- user defined function to search lat/lng information

In [5]:
def search(target):
    lat = []
    lng = []
    for i in range(len(target['key'])):
        for rep in range(0, 25):
            if rep == 0:
                url = (search_url+str(city_coord)[1:-1]+search_range+target['key'][i]+key).replace(' ','')
                tmp = urlopen(url).readlines()
            else:
                url = (search_url+str(city_coord)[1:-1]+search_range+target['key'][i]+page+key).replace(' ','')
                tmp = urlopen(url).readlines()
            
            page = '&pagetoken='+str(tmp[2])[26:-5]
            
            for k in range(len(tmp)):
                if 'location' in str(tmp[k]):
                    lat_tmp = float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+1])))
                    lng_tmp = float(re.sub('[a-z\:\\\,\'" ]','',str(tmp[k+2])))
                    lat.append(lat_tmp)
                    lng.append(lng_tmp)
    
    target['lat'] = lat
    target['lng'] = lng

    
search_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='
search_range = '&radius=20000&name=' #radius in metre up to 50,000m

In [15]:
def vmap(target, type = 'heatmap', radius = 40, size = 70):
        gmap = gmplot.GoogleMapPlotter(center_lat = city_coord[0], center_lng = city_coord[1], zoom = 12, apikey=key)
        if type == 'heatmap':
            gmap.heatmap(target['lat'], target['lng'], radius = radius)
        elif type == 'scatter':
            gmap.scatter(target['lat'], target['lng'], size = size, c = 'cornflowerblue', marker = False)
        gmap.draw(target['id']+'_map.html')
        return IFrame(src=target['id']+'_map.html', width = 1000, height = 600)

In [11]:
search(restaurant)
vmap(restaurant)

In [12]:
search(pioneer)
vmap(pioneer)

In [17]:
search(tourist)
vmap(tourist, type = 'scatter', size =100)

In [18]:
search(hang_out)
vmap(hang_out)

In [20]:
overall = {'id': 'overall'}
overall['lat'] = restaurant['lat'] + pioneer['lat'] + tourist['lat'] + hang_out['lat']
overall['lng'] = restaurant['lng'] + pioneer['lng'] + tourist['lng'] + hang_out['lng']

vmap(overall)